# Neural network model
### In this notebook, we create the neural network model with our cleaned data.

In [6]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [7]:
df = pd.read_pickle("matches_clean.pkl")
df

,match,teams,score,ht_score,date,formations,players,fifa_ids,closest_update,second_closest_update,...,normalized_form,home_team,PS_probs,B365_odds,BW_odds,IW_odds,PS_odds,WH_odds,VC_odds,max_odds
0,38313,"[Man Utd, Leicester]","[2, 1]","[1, 0]",2018-08-10,"[4-3-3, 4-2-3-1]","[[David de Gea, Goalkeeper, 4330], [Luke Shaw,...","[193080, 205988, 184392, 221660, 225508, 20929...",2018-07-19,2018-07-19,...,"[0.44, 0.44]",Man Utd,"[0.6329, 0.2545, 0.1333]","[1.57, 3.9, 7.5]","[1.53, 4.0, 7.5]","[1.55, 3.8, 7.0]","[1.58, 3.93, 7.5]","[1.57, 3.8, 6.0]","[1.57, 4.0, 7.0]","[1.58, 4.0, 7.5]"
1,38309,"[Bournemouth, Cardiff]","[2, 0]","[1, 0]",2018-08-11,"[4-4-2, 4-1-4-1]","[[Asmir Begovic, Goalkeeper, 2537], [Steve Coo...","[172723, 193011, 190885, 208920, 169638, 20780...",2018-07-19,2018-07-19,...,"[0.44, 0.44]",Bournemouth,"[0.5291, 0.2755, 0.2183]","[1.9, 3.6, 4.5]","[1.9, 3.4, 4.4]","[1.9, 3.5, 4.1]","[1.89, 3.63, 4.58]","[1.91, 3.5, 4.0]","[1.87, 3.6, 4.75]","[1.91, 3.63, 4.75]"
2,38310,"[Fulham, Crystal Palace]","[0, 2]","[0, 1]",2018-08-11,"[4-3-3, 4-4-2]","[[Fabri, Goalkeeper, 19771], [Calum Chambers, ...","[177723, 205989, 200778, 203505, 192725, 17531...",2018-07-19,2018-07-19,...,"[0.44, 0.44]",Fulham,"[0.4, 0.289, 0.3333]","[2.5, 3.4, 3.0]","[2.45, 3.3, 2.95]","[2.4, 3.3, 2.95]","[2.5, 3.46, 3.0]","[2.45, 3.3, 2.8]","[2.5, 3.4, 3.0]","[2.5, 3.46, 3.0]"
3,38311,"[Huddersfield, Chelsea]","[0, 3]","[0, 2]",2018-08-11,"[3-5-1-1, 4-3-3]","[[Ben Hamer, Goalkeeper, 3183], [Christopher S...","[170008, 200607, 183491, 203485, 183546, 19495...",2018-07-19,2018-07-19,...,"[0.44, 0.44]",Huddersfield,"[0.156, 0.2488, 0.6173]","[6.5, 4.0, 1.61]","[6.25, 3.9, 1.57]","[6.2, 4.0, 1.55]","[6.41, 4.02, 1.62]","[5.8, 3.9, 1.57]","[6.5, 4.0, 1.62]","[6.5, 4.02, 1.62]"
4,38314,"[Newcastle, Spurs]","[1, 2]","[1, 2]",2018-08-11,"[4-4-1-1, 4-3-2-1]","[[Martin Dúbravka, Goalkeeper, 6451], [Ciaran ...","[220407, 183129, 203487, 204355, 212722, 18916...",2018-07-19,2018-07-19,...,"[0.44, 0.44]",Newcastle,"[0.2611, 0.2801, 0.4808]","[3.9, 3.5, 2.04]","[3.8, 3.5, 2.0]","[3.7, 3.35, 2.05]","[3.83, 3.57, 2.08]","[3.8, 3.2, 2.05]","[3.9, 3.4, 2.1]","[3.9, 3.57, 2.1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205,93633,"[Crystal Palace, Man City]","[2, 4]","[1, 1]",2024-04-06,"[3-4-3, ]","[[Dean Henderson, Goalkeeper, 13988], [Joel Wa...","[233306, 186392, 213991, 224221, 259240, 23764...",2024-03-27,2024-03-20,...,"[0.33, 0.73]",Crystal Palace,"[0.094, 0.1789, 0.7634]","[9.5, 5.25, 1.33]","[9.0, 5.5, 1.32]","[nan, nan, nan]","[10.64, 5.59, 1.31]","[10.0, 5.5, 1.29]","[11.0, 5.5, 1.29]","[11.0, 5.59, 1.33]"
2206,93634,"[Everton, Burnley]","[1, 0]","[1, 0]",2024-04-06,"[4-4-1-1, 4-4-2]","[[Jordan Pickford, Goalkeeper, 4640], [Séamus ...","[204935, 180216, 247649, 202695, 244380, 24365...",2024-03-27,2024-03-20,...,"[0.13, 0.4]",Everton,"[0.5952, 0.2392, 0.2004]","[1.65, 4.0, 5.25]","[1.65, 4.0, 5.0]","[nan, nan, nan]","[1.68, 4.18, 4.99]","[1.65, 4.0, 4.8]","[1.62, 3.8, 5.25]","[1.68, 4.18, 5.25]"
2207,93635,"[Fulham, Newcastle]","[0, 1]","[0, 0]",2024-04-06,"[4-2-3-1, 4-3-3]","[[Bernd Leno, Goalkeeper, 4985], [Calvin Basse...","[192563, 241436, 222104, 222501, 229348, 20845...",2024-03-27,2024-03-20,...,"[0.47, 0.47]",Fulham,"[0.4255, 0.2584, 0.3497]","[2.38, 3.8, 2.8]","[2.37, 3.75, 2.7]","[nan, nan, nan]","[2.35, 3.87, 2.86]","[2.3, 3.9, 2.7]","[2.3, 3.7, 2.7]","[2.38, 3.9, 2.86]"
2208,93636,"[Luton, Bournemouth]","[2, 1]","[0, 0]",2024-04-06,"[3-4-2-1, 4-2-3-1]","[[Thomas Kaminski, Goalkeeper, 5844], [Teden M...","[188400, 253465, 221456, 253052, 202464, 25248...",2024-03-27,2024-03-20,...,"[0.13, 0.87]",Luton,"[0.274, 0.2433, 0.5181]","[3.75, 3.8, 1.91]","[3.7, 4.0, 1.85]","[nan, nan, nan]","[3.65, 4.11, 1.93]","[3.6, 4.2, 1.85]","[3.6, 3.8, 1.87]","[3.75, 4.2, 1.93]"


In [8]:
# Create features vectors
all_features = []
for _, row in df.iterrows():
    match_features = row['normalized_form'] + row['normalized_ratings'] + list(row['teams_bool'])
    all_features.append(match_features)

df['features'] = all_features

### Training on 1X2 outcome

In [101]:
# Set features to features column, and labels to outcome column
features = np.array(list(df['features']))
labels = np.array(list(df['outcome']))

### Training on bookmaker probabilities

In [9]:
# Set features to features column, and labels to Pinnacle probabilities column
features = np.array(list(df['features']))
labels = np.array(list(df['PS_probs']))

## Create model

In [11]:
# Split training and testing data
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=1)

In [67]:
# Define neural network
model = tf.keras.Sequential([
    # Set input layer and input vector shape.
    tf.keras.Input(shape=(len(df['features'][0]),)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3)
])

In [68]:
# Set optimizer algorithm and loss function
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.CategoricalCrossentropy()

# Compile model
model.compile(optimizer=optimizer, loss=loss, metrics='accuracy')

In [69]:
# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
104/104 [==============================] - 2s 9ms/step - loss: 11.4711 - accuracy: 0.6017
Epoch 2/10
104/104 [==============================] - 1s 9ms/step - loss: 12.6117 - accuracy: 0.6144
Epoch 3/10
104/104 [==============================] - 1s 10ms/step - loss: 12.6117 - accuracy: 0.6144
Epoch 4/10
104/104 [==============================] - 1s 9ms/step - loss: 12.6117 - accuracy: 0.6144
Epoch 5/10
104/104 [==============================] - 1s 9ms/step - loss: 12.6117 - accuracy: 0.6144
Epoch 6/10
104/104 [==============================] - 1s 9ms/step - loss: 12.6117 - accuracy: 0.6144
Epoch 7/10
104/104 [==============================] - 1s 10ms/step - loss: 12.6117 - accuracy: 0.6144
Epoch 8/10
104/104 [==============================] - 1s 9ms/step - loss: 12.6117 - accuracy: 0.6144
Epoch 9/10
104/104 [==============================] - 1s 9ms/step - loss: 12.6117 - accuracy: 0.6144
Epoch 10/10
104/104 [==============================] - 1s 11ms/step - loss: 12.6117 - acc

In [61]:
# Evaluate model accuracy on test data
model.evaluate(x_test, y_test, verbose=2)

18/18 - 0s - loss: 12.5570 - accuracy: 0.0000e+00 - 258ms/epoch - 14ms/step


[12.557034492492676, 0.0]

In [62]:
# Make model to transform results into probabilities
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [63]:
# Normalize model results and transform into probabilities. Works only if all model predictions are positive
def normalize_results(array):
    normalized_array = []
    for nested_array in array:
        sum = np.sum(nested_array)
        normalized_nested_array = [nested_array[0]/sum, nested_array[1]/sum, nested_array[2]/sum]
        normalized_array.append(normalized_nested_array)
    return np.array(normalized_array)

In [64]:
model(x_test[:10])

<tf.Tensor: shape=(10, 3), dtype=float32, numpy=
array([[-2.6406527 ,  4.7158413 , -0.41479924],
       [-1.8868097 ,  4.466511  , -0.38469014],
       [-1.7183982 ,  4.3274612 , -0.41037658],
       [-2.009974  ,  4.7613325 , -0.4135862 ],
       [-2.4565363 ,  5.52602   , -0.39058033],
       [-2.3728073 ,  5.4109516 , -0.590493  ],
       [-1.8162886 ,  4.6705446 , -0.279731  ],
       [-2.1656585 ,  4.93085   , -0.32013854],
       [-2.3427587 ,  5.7150717 , -0.41180018],
       [-2.383572  ,  5.6290526 , -0.5446935 ]], dtype=float32)>

In [65]:
probability_model(x_test[:10])

<tf.Tensor: shape=(10, 3), dtype=float32, numpy=
array([[6.3427771e-04, 9.9349147e-01, 5.8742887e-03],
       [1.7244705e-03, 9.9053055e-01, 7.7449395e-03],
       [2.3415945e-03, 9.8899722e-01, 8.6612031e-03],
       [1.1384513e-03, 9.9324304e-01, 5.6184554e-03],
       [3.4033254e-04, 9.9697351e-01, 2.6861907e-03],
       [4.1524300e-04, 9.9711668e-01, 2.4680367e-03],
       [1.5103694e-03, 9.9146867e-01, 7.0210434e-03],
       [8.2299492e-04, 9.9396628e-01, 5.2107032e-03],
       [3.1582333e-04, 9.9750620e-01, 2.1779547e-03],
       [3.3045621e-04, 9.9759114e-01, 2.0783979e-03]], dtype=float32)>

In [66]:
normalize_results(model(x_test[:10]))

array([[-1.5903815 ,  2.8402019 , -0.24982046],
       [-0.8595901 ,  2.0348468 , -0.1752566 ],
       [-0.78155667,  1.9682028 , -0.18664624],
       [-0.85978174,  2.0366964 , -0.17691466],
       [-0.91699326,  2.0627918 , -0.1457986 ],
       [-0.96942204,  2.210671  , -0.24124883],
       [-0.7054849 ,  1.8141383 , -0.10865344],
       [-0.8857306 ,  2.0166638 , -0.13093317],
       [-0.7913354 ,  1.9304329 , -0.13909759],
       [-0.8825472 ,  2.0842268 , -0.20167956]], dtype=float32)

In [42]:
y_test[:10]

array([[0.4115, 0.3247, 0.2857],
       [0.4484, 0.2793, 0.3003],
       [0.4049, 0.304 , 0.3135],
       [0.3663, 0.3106, 0.3497],
       [0.1391, 0.2247, 0.6667],
       [0.7299, 0.1953, 0.1002],
       [0.4367, 0.3003, 0.2849],
       [0.3876, 0.3049, 0.3356],
       [0.0803, 0.152 , 0.8   ],
       [0.1008, 0.1984, 0.7353]])

In [18]:
x_test

array([[0.4 , 0.4 , 0.6 , ..., 0.  , 0.  , 0.  ],
       [0.47, 0.07, 0.54, ..., 0.  , 0.  , 0.  ],
       [0.33, 0.27, 0.52, ..., 0.  , 0.  , 0.  ],
       ...,
       [0.27, 0.07, 0.62, ..., 0.  , 1.  , 0.  ],
       [0.33, 0.6 , 0.62, ..., 0.  , 0.  , 0.  ],
       [0.27, 0.67, 0.3 , ..., 0.  , 0.  , 0.  ]])

## Predict last 100 matches
Here we predict the odds of the last 100 matches, using all previous matches as training data. With a fictional bankroll, we simulate bets.

In [56]:
# Split data
test = df.iloc[-100:, :] # 100 rows
train = df.iloc[:-100, :] # 2110 rows
x_test = np.array(test['features'].tolist())
x_train = np.array(train['features'].tolist())
y_test = np.array(test['PS_probs'].tolist())
y_train = np.array(train['PS_probs'].tolist())

### Predict last 500

In [160]:
# Split data
test = df.iloc[-500:, :] # 500 rows
train = df.iloc[:-500, :] # 1710 rows
x_test = np.array(test['features'].tolist())
x_train = np.array(train['features'].tolist())
y_test = np.array(test['PS_probs'].tolist())
y_train = np.array(train['PS_probs'].tolist())

In [161]:
# Define neural network
model = tf.keras.Sequential([
    # Set input layer and input vector shape.
    tf.keras.Input(shape=(len(df['features'][0]),)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3)
])

In [162]:
# Set optimizer algorithm and loss function
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.CategoricalCrossentropy()

# Compile model
model.compile(optimizer=optimizer, loss=loss, metrics='accuracy')

In [163]:
# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
107/107 [==============================] - 1s 4ms/step - loss: 1.9580 - accuracy: 0.5199
Epoch 2/10
107/107 [==============================] - 0s 4ms/step - loss: 1.0682 - accuracy: 0.7573
Epoch 3/10
107/107 [==============================] - 0s 4ms/step - loss: 1.0285 - accuracy: 0.8579
Epoch 4/10
107/107 [==============================] - 0s 4ms/step - loss: 1.0137 - accuracy: 0.8696
Epoch 5/10
107/107 [==============================] - 0s 4ms/step - loss: 1.0110 - accuracy: 0.8819
Epoch 6/10
107/107 [==============================] - 0s 4ms/step - loss: 1.0044 - accuracy: 0.8877
Epoch 7/10
107/107 [==============================] - 0s 4ms/step - loss: 1.0033 - accuracy: 0.8924
Epoch 8/10
107/107 [==============================] - 0s 4ms/step - loss: 1.0023 - accuracy: 0.8854
Epoch 9/10
107/107 [==============================] - 0s 3ms/step - loss: 1.0005 - accuracy: 0.8883
Epoch 10/10
107/107 [==============================] - 0s 4ms/step - loss: 0.9996 - accuracy: 0.8953

In [164]:
# Evaluate model accuracy on test data
model.evaluate(x_test, y_test)

16/16 [==============================] - 0s 3ms/step - loss: 1.0217 - accuracy: 0.8500


[1.021660566329956, 0.8500000238418579]

In [165]:
model(x_test[:5])

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[1.9314909 , 1.4893715 , 1.7027295 ],
       [1.6502712 , 1.2084742 , 1.2433467 ],
       [2.0154066 , 0.99439347, 0.8790478 ],
       [1.1964849 , 1.0655597 , 1.6617013 ],
       [1.6044319 , 1.1410657 , 1.0705498 ]], dtype=float32)>

In [166]:
normalize_results(model(x_test[:5]))

array([[0.37697986, 0.29068896, 0.3323312 ],
       [0.4022999 , 0.2945995 , 0.30310065],
       [0.5182529 , 0.25570387, 0.22604325],
       [0.30493435, 0.27156696, 0.4234987 ],
       [0.42044336, 0.29901767, 0.28053892]], dtype=float32)

In [167]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0.35768166, 0.28943574, 0.3528826 ],
       [0.43771836, 0.2803515 , 0.28193015],
       [0.5952882 , 0.19065107, 0.2140607 ],
       [0.34417292, 0.24526969, 0.41055736],
       [0.4156307 , 0.29324713, 0.29112217]], dtype=float32)>

In [168]:
y_test[:5]

array([[0.2212, 0.2801, 0.5208],
       [0.277 , 0.2695, 0.4762],
       [0.5291, 0.2577, 0.2381],
       [0.3378, 0.2941, 0.3891],
       [0.5495, 0.274 , 0.2   ]])

In [169]:
# Set probabilities from model results using normalize_results()
#test['model_1_probs'] = probability_model(x_test).numpy().tolist()
test['model_1_probs'] = normalize_results(model(x_test)).tolist()
test['model_1_odds'] = test['model_1_probs'].apply(lambda x: [1/elem for elem in x])
# Round odds
test['model_1_odds'] = test['model_1_odds'].apply(lambda x: [round(elem, 2) for elem in x])

C:\Users\victo\AppData\Local\Temp\ipykernel_19860\820553446.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['model_1_probs'] = normalize_results(model(x_test)).tolist()
C:\Users\victo\AppData\Local\Temp\ipykernel_19860\820553446.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['model_1_odds'] = test['model_1_probs'].apply(lambda x: [1/elem for elem in x])
C:\Users\victo\AppData\Local\Temp\ipykernel_19860\820553446.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [77]:
# Last 100

test.reset_index(drop=True, inplace=True)
test.to_csv("test_100.csv", index=False)
test.to_pickle("test_100.pkl")

In [93]:
# Last 500

test.reset_index(drop=True, inplace=True)
test.to_csv("test_500.csv", index=False)
test.to_pickle("test_500.pkl")

We will simulate bets in the notebook "simulations.ipynb"

In [170]:
test.reset_index(drop=True, inplace=True)
test.to_pickle("test_500_2.pkl")

In [147]:
test

,match,teams,score,ht_score,date,formations,players,fifa_ids,closest_update,second_closest_update,...,B365_odds,BW_odds,IW_odds,PS_odds,WH_odds,VC_odds,max_odds,features,model_1_probs,model_1_odds
0,75113,"[Crystal Palace, Newcastle]","[0, 0]","[0, 0]",2023-01-21,"[4-2-3-1, 4-3-3]","[[Vicente Guaita, Goalkeeper, 7548], [Marc Gué...","[189690, 241159, 250954, 186392, 240947, 23579...",2023-01-18,2023-01-13,...,"[4.4, 3.5, 1.85]","[4.1, 3.5, 1.88]","[4.3, 3.5, 1.9]","[4.52, 3.57, 1.92]","[4.33, 3.5, 1.83]","[4.4, 3.4, 1.83]","[4.52, 3.57, 1.92]","[0.27, 0.73, 0.58, 0.56, 0.48, 0.48, 0.52, 0.5...","[0.35768166184425354, 0.2894357442855835, 0.35...","[2.8, 3.45, 2.83]"
1,75116,"[Leicester, Brighton]","[2, 2]","[1, 1]",2023-01-21,"[4-4-2, 4-2-3-1]","[[Danny Ward, Goalkeeper, 4522], [Daniel Amart...","[207998, 212883, 222501, 254470, 235036, 19886...",2023-01-18,2023-01-13,...,"[3.6, 3.5, 2.05]","[3.5, 3.6, 2.05]","[3.45, 3.6, 2.1]","[3.61, 3.71, 2.1]","[3.5, 3.5, 2.05]","[3.5, 3.5, 2.0]","[3.61, 3.71, 2.1]","[0.2, 0.6, 0.5, 0.52, 0.56, 0.48, 0.56, 0.48, ...","[0.43771836161613464, 0.2803514897823334, 0.28...","[2.28, 3.57, 3.55]"
2,75117,"[Liverpool, Chelsea]","[0, 0]","[0, 0]",2023-01-21,"[4-3-3, 3-4-2-1]","[[Alisson Becker, Goalkeeper, 20559], [Joe Gom...","[212831, 225100, 138412, 259677, 237678, 18950...",2023-01-18,2023-01-13,...,"[1.85, 3.75, 4.0]","[1.87, 3.75, 3.9]","[1.9, 3.75, 3.95]","[1.89, 3.88, 4.2]","[1.85, 3.7, 4.0]","[1.83, 3.7, 4.0]","[1.9, 3.88, 4.2]","[0.6, 0.47, 0.78, 0.62, 0.54, 0.16, 0.62, 0.72...","[0.5952882170677185, 0.190651074051857, 0.2140...","[1.68, 5.25, 4.67]"
3,75119,"[Southampton, Aston Villa]","[0, 1]","[0, 0]",2023-01-21,"[4-2-3-1, 4-4-2]","[[Gavin Bazunu, Goalkeeper, 63840], [Kyle Walk...","[243585, 227927, 244915, 230672, 225263, 26362...",2023-01-18,2023-01-13,...,"[2.88, 3.25, 2.55]","[2.8, 3.4, 2.45]","[2.85, 3.3, 2.5]","[2.96, 3.4, 2.57]","[2.8, 3.3, 2.5]","[2.8, 3.2, 2.5]","[2.96, 3.4, 2.57]","[0.2, 0.67, 0.42, 0.56, 0.54, 0.46, 0.56, 0.32...","[0.3441729247570038, 0.24526968598365784, 0.41...","[2.91, 4.08, 2.44]"
4,75120,"[West Ham, Everton]","[2, 0]","[2, 0]",2023-01-21,"[3-4-2-1, 3-4-2-1]","[[Lukasz Fabianski, Goalkeeper, 3309], [Nayef ...","[164835, 244749, 204311, 183855, 210736, 23392...",2023-01-18,2023-01-13,...,"[1.8, 3.75, 4.75]","[1.77, 3.7, 4.5]","[1.8, 3.65, 4.6]","[1.82, 3.65, 5.0]","[1.78, 3.5, 4.75]","[1.73, 3.5, 4.8]","[1.82, 3.75, 5.0]","[0.07, 0.07, 0.62, 0.6, 0.6, 0.56, 0.54, 0.64,...","[0.4156306982040405, 0.29324713349342346, 0.29...","[2.41, 3.41, 3.43]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,93633,"[Crystal Palace, Man City]","[2, 4]","[1, 1]",2024-04-06,"[3-4-3, ]","[[Dean Henderson, Goalkeeper, 13988], [Joel Wa...","[233306, 186392, 213991, 224221, 259240, 23764...",2024-03-27,2024-03-20,...,"[9.5, 5.25, 1.33]","[9.0, 5.5, 1.32]","[nan, nan, nan]","[10.64, 5.59, 1.31]","[10.0, 5.5, 1.29]","[11.0, 5.5, 1.29]","[11.0, 5.59, 1.33]","[0.33, 0.73, 0.58, 0.46, 0.56, 0.58, 0.46, 0.5...","[0.08131076395511627, 0.11212613433599472, 0.8...","[12.3, 8.92, 1.24]"
496,93634,"[Everton, Burnley]","[1, 0]","[1, 0]",2024-04-06,"[4-4-1-1, 4-4-2]","[[Jordan Pickford, Goalkeeper, 4640], [Séamus ...","[204935, 180216, 247649, 202695, 244380, 24365...",2024-03-27,2024-03-20,...,"[1.65, 4.0, 5.25]","[1.65, 4.0, 5.0]","[nan, nan, nan]","[1.68, 4.18, 4.99]","[1.65, 4.0, 4.8]","[1.62, 3.8, 5.25]","[1.68, 4.18, 5.25]","[0.13, 0.4, 0.64, 0.5, 0.54, 0.58, 0.52, 0.48,...","[0.5991236567497253, 0.23201294243335724, 0.16...","[1.67, 4.31, 5.92]"
497,93635,"[Fulham, Newcastle]","[0, 1]","[0, 0]",2024-04-06,"[4-2-3-1, 4-3-3]","[[Bernd Leno, Goalkeeper, 4985], [Calvin Basse...","[192563, 241436, 222104, 222501, 229348, 20845...",2024-03-27,2024-03-20,...,"[2.38, 3.8, 2.8]","[2.37, 3.75, 2.7]","[nan, nan, nan]","[2.35, 3.87, 2.86]","[2.3, 3.9, 2.7]","[2.3, 3.7, 2.7]","[2.38, 3.9, 2.86]","[0.47, 0.47, 0.64, 0.5, 0.52, 0.52, 0.56, 0.54...","[0.4422